# Examples from (arxiv) Appendix B

In this notebook we are going to show the automated computations to produce the results on Appendix B on the arXiv version of [this paper](https://arxiv.org/abs/2202.05550). These examples explored other sequences obtained from OEIS and have several identities we could prove. 

These examples were taken through exploration from [OEIS](https://oeis.org/). We encourage the reader to see the script `oeis_information_getter.py` in this folder to see the exploration technique and the sequences (which include those shown in this notebook) obtained from this database.

In [1]:
%display latex
from alive_progress import alive_bar
from pseries_basis import *
from pseries_basis.ore import *
from pseries_basis.io import *
from ore_algebra import *
# Creating the objects used in the notebook
OE, (x, E) = get_recurrence_algebra("x", "E")
B = BinomialBasis()

In [2]:
; B

Binomial basis (x) choose n

## The example [A007317](https://oeis.org/A007317)

This sequence (the direct binomial transform of the Catalan numbers) starts with the terms $1, 2, 5, 15, 51, 188,\ldots$ and has the following recurrence:
$$(n+3)a_{n+2} = (6n+10)a_{n+1} - 5(n+1)a_n.$$

In [25]:
L007317 = (x+3)*E^2 - (6*x+10)*E + 5*(x+1) ## The first formula in OEIS is wrong TODO: Start fixing from here
A007317 = solution(L007317, [1,2,5,15])

First, we can check that this, indeed the binomial transform of the Catalan numbers. For doing so, we compute a sequence $b_k$ such that:
$$a_n = \sum_{k=0}^n b_k \binom{n}{k},$$
obtaining the following recurrence for $b_n$:

In [26]:
equ_b = ass_recurrence(L007317, B, OE); show(equ_b)
seq_b = solution(equ_b,get_converted_init(A007317, B, required_init(equ_b))) 

(x + 3)*E^2 + (-3*x - 4)*E - 4*x - 2

On the other hand, the Catalan numbers satisfies the following recurrence:


In [27]:
equ_cat = (x+2)*E - 2*(2*(x+1)-1); equ_cat

(x + 2)*E - 4*x - 2

Using closure properties, we can check that $b_n$ is, indeed, the sequence of Catalan numbers:

In [28]:
d = required_init(equ_b.lclm(equ_cat))
print(f"We need to check equality up to {d} elements")
all(seq_b(i) == catalan_number(i) for i in range(d+1))

We need to check equality up to 4 elements


True

##### Double sum part
Let us describe the values of $a_n$ as a double sum of binomials of the form:
$$a_n = \sum_{k=0}^n \sum_{l=0}^k c_l \binom{k}{l}\binom{n}{k}.$$
Using the method `recurrence` in the package `pseries_basis` we can get the new equation for $c_k$:

In [29]:
equ_d = ass_recurrence(L007317, B, OE)
seq_d = solution(equ_d,get_converted_init(A007317, B, required_init(equ_d))) 
equ_c = ass_recurrence(equ_d, B, OE); show(equ_c)
seq_c = solution(equ_c,get_converted_init(seq_d, B, required_init(equ_c))) 
[seq_c(i) for i in range(10)]

(x + 4)*E^3 + 2*E^2 + (-7*x - 10)*E - 6*x - 6

[1, 0, 1, 1, 3, 6, 15, 36, 91, 232]

We can try to see if this new sequence appear in OEIS. In fact, id does and it provides a formula:

In [30]:
oeis([seq_c(i) for i in range(20)])

0: A005043: Riordan numbers: a(n) = (n-1)*(2*a(n-1) + 3*a(n-2))/(n+1).
1: A099323: Expansion of (sqrt(1+3*x) + sqrt(1-x))/(2*sqrt(1-x)).
2: A174297: First column of A174295.

We have three possible candidates for our sequence $(c_n)$. Let us check the first (Riordan numbers) using closure properties again:

In [31]:
riordan = oeis([seq_c(i) for i in range(20)])[0]
equ_riordan = (x+3)*E^2 - (x+1)*2*E - (x+1)*3
d = required_init(equ_c.lclm(equ_riordan))
print(f"We need to check equality up to {d} elements")
[seq_c(i) for i in range(d+1)] == list(riordan.first_terms(d+1))

We need to check equality up to 5 elements


True

Hence, we conclude that the sequence $a_n$ that satisfies the recurrence
$$(n+3)a_{n+2} = (6n+10)a_{n+1} - 5(n+1)a_n,$$
can be written by:
$$a_n = \sum_{k=0}^n c_k \binom{n}{k} = \sum_{k=0}^n \sum_{l=0}^k R_l \binom{k}{l} \binom{n}{k}.$$

## The example [A000172](https://oeis.org/A000172)

This sequence (also known as Franel numbers) starts with the terms $1, 2, 10, 56, 346, 2252,\ldots$ and satisfies the following recurrence:
$$(n + 1)^2a_{n+1} = (7n^2 + 7n + 2)a_n + 8n^2a_{n-1},\text{ for $n > 1$}$$


In [32]:
L000172 = E*((x+2)^2*E^2 - (7*(x+1)^2 + 7*(x+1) + 2)*E - 8*(x+1)^2)
A000172 = solution(L000172, [1,3,10,56])

In [33]:
[A000172(i) for i in range(10)]

[1, 3, 10, 56, 346, 2252, 15184, 104960, 739162, 5280932]

First of all, we can check that this sequence $a_n$ satisfies the following formula:
$$a_n = \sum_{k=0}^n \binom{n}{k}^3,$$
by using `ProductBasis` within the package, we can obtain a recurrence for a sequence $b_k$ such that:
$$a_n = \sum_{k=0}^n b_k \binom{n}{k}^3$$
using the following piece of code:

In [34]:
B3 = ProductBasis([B,B,B], ends=["E"]); B3
M = B3.recurrence(L000172)
fM = [B3.remove_Sni(M.coefficient((i,0))) for i in range(3)]
new_eq = fM[0].gcrd(*fM[1:])
new_eq

Sn - 1

Hence, the desired formula with the cube of the binomial holds.

##### Double sum part
Let us describe the values of $a_n$ as a double sum of binomials of the form:
$$a_n = \sum_{k=0}^n \sum_{l=0}^k c_l \binom{k}{l}\binom{n}{k}.$$
Using the method `recurrence` in the package `pseries_basis` we can get the new equation for $c_k$:

In [35]:
equ_d = ass_recurrence(L000172, B, OE)
seq_d = solution(equ_d,get_converted_init(A000172, B, required_init(equ_d))) 
equ_c = ass_recurrence(equ_d, B, OE); show(equ_c)
seq_c = solution(equ_c,get_converted_init(seq_d, B, required_init(equ_c))) 
[seq_c(i) for i in range(10)]

(x^2 + 14*x + 49)*E^7 + (5*x^2 + 59*x + 174)*E^6 + (-17*x^2 - 224*x - 719)*E^5 + (-181*x^2 - 1857*x - 4748)*E^4 + (-536*x^2 - 4500*x - 9420)*E^3 + (-760*x^2 - 5024*x - 8232)*E^2 + (-528*x^2 - 2544*x - 2976)*E - 144*x^2 - 432*x - 288

[1, 1, 2, 24, 58, 440, 16624/9, 1548368/147, 752495/14, 1163185124/3969]

This, unfortunately, is not an integer sequence anymore. et us see if we can guess a smaller recurrence:

In [144]:
guess([seq_c(i) for i in range(50)], OE)

(152*x^4 + 1694*x^3 + 5683*x^2 + 5780*x + 3575)*E^5 + (304*x^4 + 3084*x^3 + 11240*x^2 + 17833*x + 11044)*E^4 + (-3800*x^4 - 34750*x^3 - 98985*x^2 - 97859*x - 56898)*E^3 + (-16720*x^4 - 136180*x^3 - 382386*x^2 - 458014*x - 255108)*E^2 + (-23712*x^4 - 169416*x^3 - 449292*x^2 - 549492*x - 277752)*E - 10944*x^4 - 67248*x^3 - 158904*x^2 - 170136*x - 67536

## The example [A258431](https://oeis.org/A258431)

This sequence starts with the terms $0, 1, 5, 23, 102, 443, 1898, 8054,\ldots$ and counts the **_sum over all peaks of Dyck paths of semilength n of the arithmetic mean of the x and y coordinates_**. It satisfies the following recurrence equation:
$$(n-1)a_n = (8n-10)a_{n-1}-(16n-24)a_{n-2}\text{ for $n > 2$}.$$

In [36]:
L258431 = E*((x+1)*E^2 - (8*(x+2)-10)*E + (16*(x+2)-24))
A258431 = solution(L258431, [0,1,5])

In [37]:
[A258431(i) for i in range(20)]

[0,
 1,
 5,
 23,
 102,
 443,
 1898,
 8054,
 33932,
 142163,
 592962,
 2464226,
 10209620,
 42190558,
 173962532,
 715908428,
 2941192472,
 12065310083,
 49428043442,
 202249741418]

##### Double sum part
Let us describe the values of $a_n$ as a double sum of binomials of the form:
$$a_n = \sum_{k=0}^n \sum_{l=0}^k c_l \binom{k}{l}\binom{n}{k}.$$
Using the method `recurrence` in the package `pseries_basis` we can get the new equation for $c_k$:

In [38]:
equ_d = ass_recurrence(L258431, B, OE)
seq_d = solution(equ_d,get_converted_init(A258431, B, required_init(equ_d))) 
equ_c = ass_recurrence(equ_d, B, OE); show(equ_c)
seq_c = solution(equ_c,get_converted_init(seq_d, B, required_init(equ_c))) 
[seq_c(i) for i in range(10)]

(x + 4)*E^5 + (x + 1)*E^4 + (-8*x - 26)*E^3 + (-8*x - 8)*E^2 + (16*x + 40)*E + 16*x + 16

[0, 1, 1, 5, 6, 23, 30, 102, 140, 443]

We can try to see if this new sequence appear in OEIS:

In [39]:
len(oeis([seq_c(i) for i in range(1,10)]))

0

We have no sequence from OEIS. But we can try to use `guess`to see if we have another recurrence for this sequence:

In [40]:
guess([seq_c(i) for i in range(50)], OE)

(-x - 3)*E^4 + 2*E^3 + (8*x + 16)*E^2 - 8*E - 16*x - 16

This is not very nice. However, something strange happens: the original sequence appears as a subsequence of $c_k$:

In [41]:
[seq_c(2*i-1) for i in range(10)] == [A258431(i) for i in range(10)]

True

Which shows that, for the solution to 
$$(n-1)a_n = (8n-10)a_{n-1}-(16n-24)a_{n-2}\text{ for $n > 2$}, a(0) = 0, a(1) = 1, a(2) = 5;$$
if we write this sequence with
$$a_n = \sum_{k=0}^n \sum_{l=0}^k c_l \binom{k}{l}\binom{n}{k},$$
then we conclude that:
$$c_{2n-1} = a_n\text{ for all $n \in \mathbb{N}$}.$$
This final identity can be proven by closure properties of D-finite sequences.

## The example [A032443](https://oeis.org/A032443)

This sequence starts with the terms $1, 3, 11, 42, 163, 638, 2510, 9908,\ldots$ and has the following formula:
$$a_n = \sum_{i=0}^n \binom{2n}{i},$$
satisfying the following recurrence:
$$na_n + 2(-4n+3)a_{n-1} + 8(2n-3)a_{n-2} = 0$$

In [42]:
L032443 = (x+2)*E^2 + 2*(-4*(x+2)+3)*E + 8*(2*(x+2) - 3)
A032443 = solution(L032443, [1,3,11,42])

##### Double sum part
Let us describe the values of $a_n$ as a double sum of binomials of the form:
$$a_n = \sum_{k=0}^n \sum_{l=0}^k c_l \binom{k}{l}\binom{n}{k}.$$
Using the method `recurrence` in the package `pseries_basis` we can get the new equation for $c_k$:

In [43]:
equ_d = ass_recurrence(L032443, B, OE)
seq_d = solution(equ_d,get_converted_init(A032443, B, required_init(equ_d))) 
equ_c = ass_recurrence(equ_d, B, OE); show(equ_c)
seq_c = solution(equ_c,get_converted_init(seq_d, B, required_init(equ_c))) 
[seq_c(i) for i in range(10)]

(x + 4)*E^4 + (-x - 3)*E^3 + (-6*x - 16)*E^2 + (4*x + 8)*E + 8*x + 8

[1, 1, 3, 4, 11, 16, 42, 64, 163, 256]

We can try to see if this new sequence appear in OEIS. In fact, id does and it provides a formula:

In [44]:
oeis([seq_c(i) for i in range(20)])

0: A027306: a(n) = 2^(n-1) + ((1 + (-1)^n)/4)*binomial(n, n/2).

Hence we can conclude that the following identity holds:
$$\sum_{k=0}^n \binom{2n}{k} = \sum_{k=0}^n \sum_{l=0}^k\left[ \left(2^{l-1} + \frac{1 + (-1)^l}{4}\binom{l}{l/2}\right) \binom{k}{l}\binom{n}{k}\right]$$

In [45]:
## Checking identity for i = 0,...,30
lhs = lambda n : sum(binomial(2*n, k) for k in range(n+1))
rhs = lambda n : sum(sum((2**(l-1) + (1/2*binomial(l,l/2) if l%2 == 0 else 0)) * binomial(k,l) * binomial(n,k) for l in range(k+1)) for k in range(n+1))
all(lhs(i) == rhs(i) for i in range(30))

True

### Trying some examples with no-Liouvillian solutions

We know that the convolution of $n!$ and $1/n!$ is not Liouvillian and satisfies the following recurrence:
$$(n+3)y_{n+3} - (n^2 + 6n + 10)y_{n+2} + (2n + 5)y_{n+1} - y_n  = 0.$$

Let us see what happen if we apply the binomial basis to this recurrence:

In [67]:
non_liouv = lambda n : sum(QQ(factorial(k)/factorial(n-k)) for k in range(n+1));
show([non_liouv(i) for i in range(10)])
# Recurrence for the sequence
L = (x+3)*E^3 - (x^2 + 6*x + 10)*E^2 + (2*x + 5)*E - 1
[apply_operator_to_seq(L, non_liouv)(i) for i in range(10)]

[1,
 2,
 7/2,
 26/3,
 749/24,
 8843/60,
 614227/720,
 3669289/630,
 1844279257/40320,
 10519507811/25920]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [68]:
# Getting recurrence by adding a binomial factor
L2 = ass_recurrence(L, B, OE); show(L2)
# Getting the solution of the sequence
non_liouv_bin = solution(L2,get_converted_init(non_liouv, B, 6)) 
[non_liouv_bin(i) for i in range(10)]

(x + 5)*E^5 + (-x^2 - 6*x - 9)*E^4 + (-4*x^2 - 25*x - 40)*E^3 + (-6*x^2 - 31*x - 41)*E^2 + (-4*x^2 - 16*x - 16)*E - x^2 - 3*x - 2

[1,
 1,
 1/2,
 19/6,
 253/24,
 6241/120,
 219661/720,
 10574971/5040,
 667791769/40320,
 53505430273/362880]

In [69]:
# Getting recurrence by adding a second binomial factor
L3 = ass_recurrence(L, B, OE); show(L3)
# Getting the solution of the sequence
non_liouv_bin_bin = solution(L2,get_converted_init(non_liouv_bin, B, 6)) 
[non_liouv_bin_bin(i) for i in range(10)]

(x + 5)*E^5 + (-x^2 - 6*x - 9)*E^4 + (-4*x^2 - 25*x - 40)*E^3 + (-6*x^2 - 31*x - 41)*E^2 + (-4*x^2 - 16*x - 16)*E - x^2 - 3*x - 2

[1,
 0,
 -1/2,
 11/3,
 -17/8,
 2617/120,
 56437/720,
 3045667/5040,
 189260353/40320,
 15193219561/362880]

In [74]:
guess([non_liouv_bin_bin(i) for i in range(50)], OE)

(-x - 5)*E^5 + (x^2 + 6*x + 9)*E^4 + (4*x^2 + 25*x + 40)*E^3 + (6*x^2 + 31*x + 41)*E^2 + (4*x^2 + 16*x + 16)*E + x^2 + 3*x + 2

### Trying to use guess to obtain an example

In [203]:
a = -1; b = -2; c = -3
seq = lambda n : solution(E^2 - (x-a)*(x-b)*E - (x-a)*(x-b)*(x-c), [1,1])(n)
bin_seq = lambda n : sum(binomial(n,k)*seq(k) for k in range(n+1))
bin_bin_seq = lambda n : sum(binomial(n,k)*bin_seq(k) for k in range(n+1))

In [204]:
[seq(i) for i in range(10)]

[1, 1, 8, 72, 1344, 35520, 1347840, 68544000, 4517775360, 374631505920]

In [205]:
[bin_bin_seq(i) for i in range(20)]

[1,
 3,
 16,
 140,
 2160,
 52592,
 1868416,
 90820032,
 5783000320,
 466805000960,
 46565899505664,
 5624284106058752,
 808786822879473664,
 136539590048636497920,
 26739228138809412812800,
 6012248327840046068645888,
 1538283273936198105335660544,
 444360163131793080333495369728,
 143917487573718575394901511372800,
 51937374756541740876020471330242560]

## The example [A001472](https://oeis.org/A001472)

This sequence starts with the terms $1, 1, 2, 4, 16, 56, 256, 1072, 6224, 33616, 218656,\ldots$ and counts the number of degree-$n$ permutations of order dividing 4. It satisfies a recurrence: 
$$a(0)=1, a(1)=1, a(2)=2, a(3)=4,$$
$$a(n) = a(n-1) + (n-1)a(n-2) + (n^3-6n^2+11n-6)a(n-4)$$

In [34]:
L001472 = E^4 - E^3 - (x+3)*E^2 - ((x+4)^3 - 6*(x+4)^2 + 11*(x+4) - 6)
A001472 = solution(L001472, [1,1,2,4])
[A001472(i) for i in range(11)]

[1, 1, 2, 4, 16, 56, 256, 1072, 6224, 33616, 218656]

In [35]:
seq, op = explore_example(L001472, [1,1,2,4], OE, B, B)

In [36]:
[seq(i) for i in range(20)]

[1,
 3,
 10,
 36,
 144,
 648,
 3264,
 18000,
 107088,
 682992,
 4657824,
 33763392,
 258543360,
 2082340224,
 17600910336,
 155694742272,
 1436921254656,
 13798542792960,
 137635003878912,
 1423560240073728]

This sequence has a formula given as follows:

$$a(n) = n!\sum_{k=1}^n \frac{1}{k!}\left(\sum_{j=\lfloor(4*k-n)/3\rfloor}^k\binom{k}{j} \binom{j}{n-4k+3j}(1/2)^{n-4k+3j}(1/4)^{k-j}\right)$$

In [219]:
bin_A001472 = lambda n : sum(binomial(n,k)*A001472(k) for k in range(n+1))
Lbin_001472 = guess([bin_A001472(i) for i in range(50)], OE)
bin_bin_A001472 = lambda n : sum(binomial(n,k)*bin_A001472(k) for k in range(n+1))
Lbin_bin_001472 = guess([bin_bin_A001472(i) for i in range(50)], OE)
show(L001472)
show(Lbin_001472)
print(latex(-Lbin_bin_001472).replace("x", "n"))
[bin_bin_A001472(i) for i in range(10)]

E^4 - E^3 + (-x - 3)*E^2 - x^3 - 6*x^2 - 11*x - 6

-E^4 + 2*E^3 + (x + 3)*E^2 + x^3 + 6*x^2 + 11*x + 6

E^{4} - 3 E^{3} + \left(-n - 3\right) E^{2} - n^{3} - 6 n^{2} - 11 n - 6


[1, 3, 10, 36, 144, 648, 3264, 18000, 107088, 682992]

#### Writing the example

Consider the sequence $(1,3,10,36,144,648,3264,18000,107088,682992,\ldots)$ defined as solution to the recurrence equation:
$$y_{n+4} - 3 y_{n+3} + \left(-n - 3\right) y_{n+2} - (n^{3} + 6 n^{2} + 11 n + 6)y_n = 0$$

This sequence does not show up in OEIS, but if we write it in the following form:
$$y_n = \sum_{k=0}^n c_k \binom{n}{k},$$
then the sequence $(c_n)_n$ also satisfies a recurrence equation given by:

In [229]:
equ_y = -Lbin_bin_001472
seq_y = solution(equ_y, [1,3,10,36, 144])
equ_c = ass_recurrence(equ_y, B, OE); show(equ_c)
seq_c = solution(equ_c,get_converted_init(seq_y, B, 7)) 
[seq_c(i) for i in range(10)]

E^7 + E^6 + (-x - 9)*E^5 + (-3*x - 20)*E^4 + (-x^3 - 15*x^2 - 77*x - 134)*E^3 + (-3*x^3 - 36*x^2 - 142*x - 183)*E^2 + (-3*x^3 - 27*x^2 - 78*x - 72)*E - x^3 - 6*x^2 - 11*x - 6

[1, 2, 5, 14, 49, 202, 949, 4790, 26513, 159218]

This sequence does not appear in OEIS still. The order has gotten bigger and we still have no information. We can try to apply the transformation once again:
$$c_n = \sum_{k=0}^nd_k\binom{n}{k}.$$ 
This sequence $(d_n)_n$ is also D-finite. Let see the recurrence and solution:

In [232]:
equ_d = ass_recurrence(equ_c, B, OE); show(equ_d)
seq_d = solution(equ_d,get_converted_init(seq_c, B, 10)) 
[seq_d(i) for i in range(10)]

E^10 + 8*E^9 + (-x + 15)*E^8 + (-9*x - 42)*E^7 + (-x^3 - 24*x^2 - 224*x - 732)*E^6 + (-9*x^3 - 189*x^2 - 1377*x - 3432)*E^5 + (-33*x^3 - 594*x^2 - 3597*x - 7288)*E^4 + (-63*x^3 - 945*x^2 - 4698*x - 7712)*E^3 + (-66*x^3 - 792*x^2 - 3110*x - 3984)*E^2 + (-36*x^3 - 324*x^2 - 936*x - 864)*E - 8*x^3 - 48*x^2 - 88*x - 48

[1, 1, 2, 4, 16, 56, 256, 1072, 6224, 33616]

This sequence does appear in OEIS as the sequence [A001472](https://oeis.org/A001472). Sequence A001472 is D-finite with recurrence
$$a(n) = a(n-1) + (n-1)a(n-2) + (n^3-6n^2+11n-6)a(n-4).$$
Using closure properties, we can check if these two sequences are the same:

In [240]:
L001472 = E^4 - E^3 - (x+3)*E^2 - ((x+4)^3 - 6*(x+4)^2 + 11*(x+4) - 6)
A001472 = solution(L001472, [1,1,2,4])
diff_equ = L001472.lclm(equ_d)
diff_seq = solution(diff_equ, [A001472(i) - seq_d(i) for i in range(diff_equ.order()+5)])
show(diff_equ)
[diff_seq(i) for i in range(diff_equ.order()+1)]

E^10 + 8*E^9 + (-x + 15)*E^8 + (-9*x - 42)*E^7 + (-x^3 - 24*x^2 - 224*x - 732)*E^6 + (-9*x^3 - 189*x^2 - 1377*x - 3432)*E^5 + (-33*x^3 - 594*x^2 - 3597*x - 7288)*E^4 + (-63*x^3 - 945*x^2 - 4698*x - 7712)*E^3 + (-66*x^3 - 792*x^2 - 3110*x - 3984)*E^2 + (-36*x^3 - 324*x^2 - 936*x - 864)*E - 8*x^3 - 48*x^2 - 88*x - 48

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Sequnce A001472 has a closed form formula. Hence, we can write the solutions to our original equation
$$y_{n+4} - 3 y_{n+3} + \left(-n - 3\right) y_{n+2} - (n^{3} + 6 n^{2} + 11 n + 6)y_n = 0$$
with the following formula:
$$y_n = \sum_{k=0}^n 
    \sum_{l=0}^k 
        \left(l!\sum_{p=1}^l 
            \frac{1}{p!}\left(
                \sum_{j=\lfloor(4p-l)/3\rfloor}^p\binom{p}{j} \binom{j}{l-4p+3j}(1/2)^{l-4p+3j}(1/4)^{p-j}
            \right)
        \right)
\binom{k}{l}\binom{n}{k}$$

In [9]:
sum(primes_first_n(8)).divides(prod(primes_first_n(8)))

True